In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import pickle
import pdb

In [2]:
data = pickle.load(open('./data/restructured_dataset.pkl', 'rb'))

In [3]:
def process_label(label):
    if label[0] == '/':
        return label.split('/')[1]
    else:
        return label
def divide_to_test_and_train(features, labels, test_nodes=None, ids=None):
    if test_nodes is None:
        last_train_index = 80000
        features_train = features[:last_train_index]
        labels_train = labels[:last_train_index]
        features_test = features[last_train_index:]
        labels_test = labels[last_train_index:]
    else:
        assert(ids is not None)
        features_train = []
        labels_train = []
        features_test = []
        labels_test = []
        
        for i in range(len(features)):
            if (ids[i][0] in test_nodes) or (ids[i][1] in test_nodes):
                features_test.append(features[i])
                labels_test.append(labels[i])
            else:
                features_train.append(features[i])
                labels_train.append(labels[i])
    return features_train, labels_train, features_test, labels_test

In [4]:
def print_for_this_number_of_test_nodes(minimum_number_of_test_nodes_wanted):

    # another prototype
    features = []
    labels = []
    ids = []

    test_nodes = []
    treat_as_test_node = False
    for citing_id in data['edges'].keys():
        cited_ids = data['edges'][citing_id]
        if len(test_nodes) < minimum_number_of_test_nodes_wanted:
            treat_as_test_node = True
        if citing_id in data['features_for_id'].keys() and citing_id in data['label_for_id'].keys():
            for i in range(len(cited_ids)):
                if cited_ids[i] in data['features_for_id'].keys() and cited_ids[i] in data['label_for_id'].keys():
                    ids.append([citing_id, cited_ids[i]])
                    the_elements = [
                        data['features_for_id'][citing_id], data['features_for_id'][cited_ids[i]]
                    ]
                    the_labels = [
                        process_label(data['label_for_id'][citing_id]), process_label(data['label_for_id'][cited_ids[i]])
                    ]
                    if not (the_labels[0] == "NOLABEL" or the_labels[1] == "NOLABEL"):
                        features.append(the_elements)
                        labels.append(the_labels)
                        if treat_as_test_node:
                            test_nodes += [citing_id, cited_ids[i]]
                            test_nodes = list(set(test_nodes))
        treat_as_test_node = False

    features_train, labels_train, features_test, labels_test = divide_to_test_and_train(features, labels, test_nodes, ids)

    trainer = pycrfsuite.Trainer(verbose=False)

    for xseq, yseq in zip(features_train, labels_train):
        try:
            trainer.append(xseq, yseq)
        except:
            pdb.set_trace()

    trainer.set_params({
        'c1': 1.0,   # coefficient for L1 penalty
        'c2': 1e-3,  # coefficient for L2 penalty
        'max_iterations': 50,  # stop earlier

        # include transitions that are possible, but not observed
        'feature.possible_transitions': True
    })


    trainer.train('cora%d.crfsuite' % minimum_number_of_test_nodes_wanted)

    tagger = pycrfsuite.Tagger()
    tagger.open('cora%d.crfsuite' % minimum_number_of_test_nodes_wanted)

    from collections import Counter
    info = tagger.info()

    def print_transitions(trans_features):
        for (label_from, label_to), weight in trans_features:
            print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

    print("Top likely transitions:")
    print_transitions(Counter(info.transitions).most_common(15))

    print("\nTop unlikely transitions:")
    print_transitions(Counter(info.transitions).most_common()[-15:])

    def full_classification_report(y_true, y_pred):
        lb = LabelBinarizer()
        y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
        y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))

        tagset = set(lb.classes_) - {'O'}
        tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
        class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

        return classification_report(
            y_true_combined,
            y_pred_combined,
            labels = [class_indices[cls] for cls in tagset],
            target_names = tagset,
        )


    y_pred = [tagger.tag(xseq) for xseq in features_test]

    print(full_classification_report(labels_test, y_pred))

In [5]:
print_for_this_number_of_test_nodes(500)

Top likely transitions:
Artificial_Intelligence -> Artificial_Intelligence 3.415276
Programming -> Programming 3.032480
Data_Structures__Algorithms_and_Theory -> Data_Structures__Algorithms_and_Theory 2.791714
Human_Computer_Interaction -> Human_Computer_Interaction 2.386109
Encryption_and_Compression -> Encryption_and_Compression 2.297515
Hardware_and_Architecture -> Hardware_and_Architecture 2.150031
Information_Retrieval -> Information_Retrieval 1.930336
Networking -> Networking 1.480206
Databases -> Databases 1.412418
Data_Structures__Algorithms_and_Theory -> Artificial_Intelligence 1.031583
Programming -> Data_Structures__Algorithms_and_Theory 0.944250
Data_Structures__Algorithms_and_Theory -> Programming 0.838409
Artificial_Intelligence -> Data_Structures__Algorithms_and_Theory 0.836973
Operating_Systems -> Operating_Systems 0.786721
Artificial_Intelligence -> Programming 0.736377

Top unlikely transitions:
Artificial_Intelligence -> Networking -1.005958
Encryption_and_Compressio